## Wikidata to Wikipedia

Query Wikidata endpoint to get Wikipedia pages of (the subset of) people in education that are both in Wikidata and Wikipedia.

In [ ]:
#!pip install sparql-dataframe

import sparql_dataframe
import pandas as pd
from collections import defaultdict

SPARQL queries to get wikipedia pages URL

In [ ]:
# males with a female doctoral advisor
q1 =["Male phd students of female doctoral advisors", ''' 
SELECT DISTINCT ?article WHERE {
  ?person wdt:P184 ?advisor . ?advisor wdt:P21 wd:Q6581072  . ?person wdt:P21 wd:Q6581097 . 
  ?article schema:about ?person ; schema:inLanguage "en"}
''']

# Female doctoral advisor of males
q2 = ["Female advisors of male phd students",''' 
SELECT DISTINCT ?article WHERE {
  ?person wdt:P184 ?advisor . ?advisor wdt:P21 wd:Q6581072  . ?person wdt:P21 wd:Q6581097 . 
  ?article schema:about ?advisor ; schema:inLanguage "en"}
''']

# males with a male doctoral advisor
q3 = ["Males phd students of male advisors",''' 
SELECT DISTINCT ?article WHERE {
  ?person wdt:P184 ?advisor . ?advisor wdt:P21 wd:Q6581097  . ?person wdt:P21 wd:Q6581097 . 
  ?article schema:about ?person ; schema:inLanguage "en"}
''']

# male doctoral advisor of males
q4 = ["Male advisors of male phd students", ''' 
SELECT DISTINCT ?article WHERE {
  ?person wdt:P184 ?advisor . ?advisor wdt:P21 wd:Q6581097  . ?person wdt:P21 wd:Q6581097 . 
  ?article schema:about ?advisor ; schema:inLanguage "en"}
''']

# females with a female doctoral advisor
q5 = ["Females phd students of female advisors", ''' 
SELECT DISTINCT ?article WHERE {
  ?person wdt:P184 ?advisor . ?advisor wdt:P21 wd:Q6581072  . ?person wdt:P21 wd:Q6581072 . 
  ?article schema:about ?person ; schema:inLanguage "en"}
''']

# Female doctoral advisor of females
q6 = ["Female advisors of female phd students", ''' 
SELECT DISTINCT ?article WHERE {
  ?person wdt:P184 ?advisor . ?advisor wdt:P21 wd:Q6581072  . ?person wdt:P21 wd:Q6581072 . 
  ?article schema:about ?advisor ; schema:inLanguage "en"}
''']

# females with a male doctoral advisor
q7 = ["Female phd students of male advisors",''' 
SELECT DISTINCT ?article WHERE {
  ?person wdt:P184 ?advisor . ?advisor wdt:P21 wd:Q6581097  . ?person wdt:P21 wd:Q6581072 . 
  ?article schema:about ?person ; schema:inLanguage "en"}
''']

# male doctoral advisor of females
q8 = ["Male advisors of female phd students",''' 
SELECT DISTINCT ?article WHERE {
  ?person wdt:P184 ?advisor . ?advisor wdt:P21 wd:Q6581097  . ?person wdt:P21 wd:Q6581072 . 
  ?article schema:about ?advisor ; schema:inLanguage "en"}
''']

# Male phd students of women
q9 = ["Male phd students of female advisors",''' 
SELECT DISTINCT ?article WHERE {
  ?advisor wdt:P185 ?person. ?advisor wdt:P21 wd:Q6581072  . ?person wdt:P21 wd:Q6581097 .
  ?article schema:about ?person ; schema:inLanguage "en"}
''']

# Female phd students of women
q10 = ["Female phd students of female advisors",''' 
SELECT DISTINCT ?article WHERE {
  ?advisor wdt:P185 ?person. ?advisor wdt:P21 wd:Q6581072  . ?person wdt:P21 wd:Q6581072 .
  ?article schema:about ?person ; schema:inLanguage "en"}
''']

# Women advisors of women
q11 = ["Female advisors of female phd students",''' 
SELECT DISTINCT ?article WHERE {
  ?advisor wdt:P185 ?person. ?advisor wdt:P21 wd:Q6581072  . ?person wdt:P21 wd:Q6581072 .
  ?article schema:about ?advisor ; schema:inLanguage "en"}
''']

# Women advisors of male phd students
q12 = ["Female advisors of male phd students",''' 
SELECT DISTINCT ?article WHERE {
  ?advisor wdt:P185 ?person. ?advisor wdt:P21 wd:Q6581072  . ?person wdt:P21 wd:Q6581097 .
  ?article schema:about ?advisor ; schema:inLanguage "en"}
''']

# Male phd students of men
q13 = ["Male phd students of male advisors",''' 
SELECT DISTINCT ?article WHERE {
  ?advisor wdt:P185 ?person. ?advisor wdt:P21 wd:Q6581097  . ?person wdt:P21 wd:Q6581097 .
  ?article schema:about ?person ; schema:inLanguage "en"}
''']

# men advisors of male phd students
q14 = ["Men advisors of male phd students",''' 
SELECT DISTINCT ?article WHERE {
  ?advisor wdt:P185 ?person. ?advisor wdt:P21 wd:Q6581097  . ?person wdt:P21 wd:Q6581097 .
  ?article schema:about ?advisor ; schema:inLanguage "en"}
''']

# men advisors of female phd students
q15 = ["Men advisors of female phd students",''' 
SELECT DISTINCT ?article WHERE {
  ?advisor wdt:P185 ?person. ?advisor wdt:P21 wd:Q6581097  . ?person wdt:P21 wd:Q6581072 .
  ?article schema:about ?advisor ; schema:inLanguage "en"}
''']

# Female phd students of male advisors
q16 = ["Female phd students of male advisors",''' 
SELECT DISTINCT ?article WHERE {
  ?advisor wdt:P185 ?person. ?advisor wdt:P21 wd:Q6581097  . ?person wdt:P21 wd:Q6581072 .
  ?article schema:about ?person ; schema:inLanguage "en"}
''']

# males with a female advisor
q17 = ["Male students of female advisors",''' 
SELECT DISTINCT ?article WHERE {
  ?person wdt:P1066 ?advisor . ?advisor wdt:P21 wd:Q6581072  . ?person wdt:P21 wd:Q6581097 .
  ?article schema:about ?person ; schema:inLanguage "en"}
''']

# Female advisor of males
q18 = ["Female advisors of male students",''' 
SELECT DISTINCT ?article WHERE {
  ?person wdt:P1066 ?advisor . ?advisor wdt:P21 wd:Q6581072  . ?person wdt:P21 wd:Q6581097 .
  ?article schema:about ?advisor ; schema:inLanguage "en"}
''']

# males with a male advisor
q19 = ["Male students of male advisors",''' 
SELECT DISTINCT ?article WHERE {
  ?person wdt:P1066 ?advisor . ?advisor wdt:P21 wd:Q6581097  . ?person wdt:P21 wd:Q6581097 .
  ?article schema:about ?person ; schema:inLanguage "en"}
''']

# male advisor of males
q20 = ["Male advisors of male students",''' 
SELECT DISTINCT ?article WHERE {
  ?person wdt:P1066 ?advisor . ?advisor wdt:P21 wd:Q6581097  . ?person wdt:P21 wd:Q6581097 .
  ?article schema:about ?advisor ; schema:inLanguage "en"}
''']

# females with a female advisor
q21 = ["Female students of female advisors",''' 
SELECT DISTINCT ?article WHERE {
  ?person wdt:P1066 ?advisor . ?advisor wdt:P21 wd:Q6581072  . ?person wdt:P21 wd:Q6581072 .
  ?article schema:about ?person ; schema:inLanguage "en"}
''']

# Female advisor of females
q22 = ["Female advisors of female students",''' 
SELECT DISTINCT ?article WHERE {
  ?person wdt:P1066 ?advisor . ?advisor wdt:P21 wd:Q6581072  . ?person wdt:P21 wd:Q6581072 .
  ?article schema:about ?advisor ; schema:inLanguage "en"}
''']

# females with a male advisor
q23 = ["Female students of a male advisors",''' 
SELECT DISTINCT ?article WHERE {
  ?person wdt:P1066 ?advisor . ?advisor wdt:P21 wd:Q6581097  . ?person wdt:P21 wd:Q6581072.
  ?article schema:about ?person ; schema:inLanguage "en"}
''']

# male advisor of females
q24 = ["Male advisors of female students",''' 
SELECT DISTINCT ?article WHERE {
  ?person wdt:P1066 ?advisor . ?advisor wdt:P21 wd:Q6581097  . ?person wdt:P21 wd:Q6581072.
  ?article schema:about ?advisor ; schema:inLanguage "en"}
''']

# Male students of women
q25 = ["Male students of female advisors",''' 
SELECT DISTINCT ?article WHERE {
  ?advisor wdt:P802 ?person. ?advisor wdt:P21 wd:Q6581072  . ?person wdt:P21 wd:Q6581097 .
  ?article schema:about ?person ; schema:inLanguage "en"}
''']

# Women advisors of male students
q26 = ["Female advisors of male students",''' 
SELECT DISTINCT ?article WHERE {
  ?advisor wdt:P802 ?person. ?advisor wdt:P21 wd:Q6581072  . ?person wdt:P21 wd:Q6581097 .
  ?article schema:about ?advisor ; schema:inLanguage "en"}
''']

# Male students of men
q27 = ["Male students of male advisors",''' 
SELECT DISTINCT ?article WHERE {
  ?advisor wdt:P802 ?person. ?advisor wdt:P21 wd:Q6581097  . ?person wdt:P21 wd:Q6581097 .
  ?article schema:about ?person ; schema:inLanguage "en"}
''']

# men advisors of male students
q28 = ["Male advisors of male students",''' 
SELECT DISTINCT ?article WHERE {
  ?advisor wdt:P802 ?person. ?advisor wdt:P21 wd:Q6581097  . ?person wdt:P21 wd:Q6581097 .
  ?article schema:about ?advisor ; schema:inLanguage "en"}
''']

# men advisors of female students
q29 = ["Male advisors of female students",''' 
SELECT DISTINCT ?article WHERE {
  ?advisor wdt:P802 ?person. ?advisor wdt:P21 wd:Q6581097  . ?person wdt:P21 wd:Q6581072 .
  ?article schema:about ?advisor ; schema:inLanguage "en"}
''']

# Female students of men
q30 = ["Female students of male advisors",''' 
SELECT DISTINCT ?article WHERE {
  ?advisor wdt:P802 ?person. ?advisor wdt:P21 wd:Q6581097  . ?person wdt:P21 wd:Q6581072 .
  ?article schema:about ?person ; schema:inLanguage "en"}
''']

# Female students of female
q31 = ["Female students of female advisors",''' 
SELECT DISTINCT ?article WHERE {
  ?advisor wdt:P802 ?person. ?advisor wdt:P21 wd:Q6581072  . ?person wdt:P21 wd:Q6581072 .
  ?article schema:about ?person ; schema:inLanguage "en"}
''']

# Female advisors of female
q32 = ["Female advisors of female students", ''' 
SELECT DISTINCT ?article WHERE {
  ?advisor wdt:P802 ?person. ?advisor wdt:P21 wd:Q6581072  . ?person wdt:P21 wd:Q6581072 .
  ?article schema:about ?advisor ; schema:inLanguage "en"}
''']


Query Wikidata and create list of unique URLs

In [ ]:
endpoint = 'https://query.wikidata.org/bigdata/namespace/wdq/sparql'
queries = [q1,q2,q3,q4,q5,q6,q7,q8,q9,q10,q11,q12,q13,q14,q15,q16,q17,q18,q19,q20,q21,q22,q23,q24,q25,q26,q27,q28,q29,q30,q31,q32]

all_pages = defaultdict(list)
for q in queries:
  try:
    df = sparql_dataframe.get(endpoint, q[1], post=True)
    res= df['article'].values.tolist()
    for page in res:
      all_pages[page].append(q[0])
  except Exception as e:
    print(e)

all_pages

Save the list of wiki pages along with its classification in a json file

In [ ]:
all_pages = dict(all_pages)
import json
with open("wikipedia_pages.json", "w") as outfile:
    json.dump(all_pages, outfile)

Remove categories from results

In [ ]:
len(all_pages)
# 103.142
clean_all_pages = dict()
for k,v in all_pages.items():
  if 'Category:' not in k:
    clean_all_pages[k] = v

len(clean_all_pages)
with open("clean_wikipedia_pages.json", "w") as outfile:
    json.dump(clean_all_pages, outfile)
# 67.637

## Export wikipedia pages

In [ ]:
#!pip3 install wikipedia-api
import wikipediaapi
import os, json

def export_wikipediaapi():
  """Export html pages via Wikimedia action API 
  See: https://www.mediawiki.org/wiki/Special:MyLanguage/API:Main_page. 
  Problem: results limit"""
  wiki_wiki = wikipediaapi.Wikipedia('en')
  wiki_html = wikipediaapi.Wikipedia(
          language='en',
          extract_format=wikipediaapi.ExtractFormat.HTML)

  with open('clean_wikipedia_pages.json') as json_file:
      pages = json.load(json_file)

  for page, categories in pages.items():
    page_name = page.rsplit('/',1)[1]
    page = wiki_html.page(page_name)
    if os.path.exists('corpus/'+page_name+'.html'): # and (os.path.getsize('corpus/'+page_name+'.html')):
      pass
    else:
      try:
        html = page.text
        f= open('corpus/'+page_name+".html", "w")
        f.write(html)
        f.close()
        print(page_name)
      except Exception as e:
        print(page_name, e)

Export wiki summaries in txt via Wiki REST API

In [11]:
import requests
with open('clean_wikipedia_pages.json') as json_file:
    pages = json.load(json_file)

for page, categories in pages.items():
  page_name = page.rsplit('/',1)[1]
  
  if os.path.exists('data_wiki/'+page_name+'.json'): # and (os.path.getsize('corpus/'+page_name+'.html')):
    pass
  else:
    try:
      r = requests.get('https://en.wikipedia.org/api/rest_v1/page/summary/'+page_name)
      if r.status_code == 200:
        t = r.json()
        text = t['extract']
        f= open('data_wiki/'+page_name+".txt", "w")
        f.write(text)
        f.close()
      print(page_name)
    except Exception as e:
      print(page_name, e)

Output streaming troncato alle ultime 5000 righe.
Frieda_Belinfante
Amanda_R%C3%B6ntgen-Maier
Nancy_Dalberg
Zoia_Korvin-Krukovsky
Gabriele_M%C3%BCnter
Gabriele_M%C3%BCnter
Birgit_Cullberg
Ruth_Sch%C3%B6nthal
Olga_Gorelli
Nina_%C5%A0enk
Mansi_Barberis
Monique_M%C3%A9linand
Berta_Alves_de_Sousa
Eleni_Lambiri
Fr%C3%A9d%C3%A9rique_%C3%89milie_Auguste_O%27Connell
Anne_Tyng
Lydia_of_Thyatira
Agnes_de_Frumerie
Baladine_Klossowska
Rose_(Street_Fighter)
Khema
Jeanne_Bardey
Alice_Ambrose
Sonome
Eva_Gonzal%C3%A8s
Fanny_Falkner
Tyra_Lundgren
Ragnhild_Butensch%C3%B8n
Gira_Sarabhai
Hawkeye_(Kate_Bishop)
H%C3%A9l%C3%A8ne_Marie_Antigna
Edith_Stein
Edith_Stein
Edith_Stein
Enid_Yandell
Winifred_Hoernl%C3%A9
Maria_Rodrigo
Pepi_Weixlg%C3%A4rtner-Neutra
Isabel_Mundry
Margarete_Sch%C3%BCtte-Lihotzky
Clara_Anna_Korn
Saar_de_Swart
Karoline_Jagemann
Berthe_Morisot
Berthe_Morisot
Berthe_Morisot
Morisot,_Berthe
Laura_Netzel
Margherita_Torretta
Emma_Kissling
Katharina_Sieverding
Ella_Adayevskaya
Emilie_Juliane_of

Download

In [13]:
import shutil
name= 'data_wiki'
shutil.make_archive(name, 'zip', 'data_wiki') # make zip
from google.colab import files
files.download(name+".zip") # download from colab

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>